# The At operator: `@`

> Once you understand how to write a program get someone else to write it.
\--_Alan Perlis_

Let's look at the powerful immutable array update operator, _At_ (`@`). By "immutable" in this context we mean non-destructive -- it creates a new array, rather than mutating in place. Immutability is a good thing.

The dyadic `@` operator has a lot of moving parts, and we won't cover all possible combinations here. 

References for `@`:
* [APL Cart](https://aplcart.info/?q=%40#) has loads of examples
* The APL Cultivations covered it in [Lesson 4](https://chat.stackexchange.com/rooms/52405/conversation/lesson-4-more-apl-operators--) (amongst other things)
* Dyalog [docs](https://help.dyalog.com/latest/#Language/Primitive%20Operators/At.htm)

Our usual prelude:

In [1]:
⎕IO ← 0
]box on
]rows on

Was ON

Was OFF

`@` can take both functions and arrays as either operand. In its simplest form, the left operand specifies values, and the right operand indexes:

In [2]:
0@1 2 3⊢1 1 1 1 1 1 1

1 0 0 0 1 1 1

Recall from the introduction to operators that an operator returns a derived function. We inserted a _Right tack_ to make clear the distinction -- to stop stranding -- between the operator's right operand and the derived function's argument. We could equally well have written:

In [75]:
(0@1 2 3) 1 1 1 1 1 1 1

┌→────────────┐
│1 0 0 0 1 1 1│
└~────────────┘

We can think of the right operand as defining a selection criteria. For example, it can be a _function_ that when called with the derived function's argument returns a Boolean array:

In [76]:
'*'@{0 1 1 1 0 0 0 0 0 0 0} 'Hello world' ⍝ Right operand: function returning Boolean array

┌→──────────┐
│H***o world│
└───────────┘

The left operand is either an array providing the new values for every value selected by the right argument, or a function which will be called with each selected element to produce the replacement value. For example, let's add 5 to every even element:

In [4]:
{5+⍵}@{0=2|⍵} ⍳12

5 1 7 3 9 5 11 7 13 9 15 11

In this case, the right operand is the function `{0=2|⍵}`, which is called as such:

In [78]:
{0=2|⍵} ⍳12 ⍝ Return Boolean mask indicating the even numbers

┌→──────────────────────┐
│1 0 1 0 1 0 1 0 1 0 1 0│
└~──────────────────────┘

The selected numbers are then:

In [79]:
({0=2|⍵} ⍳12)/⍳12 ⍝ Compress

┌→───────────┐
│0 2 4 6 8 10│
└~───────────┘

which are passed to the left operand:

In [80]:
{5+⍵} 0 2 4 6 8 10 ⍝ Left operand applied to elements selected by right operand

┌→─────────────┐
│5 7 9 11 13 15│
└~─────────────┘

Finally, @ will make the substitution.

Let's write our own naive, overly verbose, partial implementation of @ for illustrative purposes. 

In [5]:
]dinput
_At_←{ ⍝ Partial @ - left and right operands must be functions, and the right arg a vector
    mutator ← ⍺⍺                   ⍝ Left operand -- function only
    selector ← ⍵⍵                  ⍝ Right operand -- function only
    data ← ⍵                       ⍝ Derived function right argument -- vector only. Note: copy
    mask ← selector data
    selection ← mask/data          ⍝ Compress the data according to boolean mask
    newvals ← mutator selection    ⍝ Mutate our selection
    (mask/data) ← newvals          ⍝ Replace with the mutated values in copy
    data                           ⍝ Return result
}

In [82]:
{5+⍵}_At_{0=2|⍵} ⍳12

┌→──────────────────────────┐
│5 1 7 3 9 5 11 7 13 9 15 11│
└~──────────────────────────┘

We can shrink that down a bit without any real loss of clarity:

In [83]:
]dinput
_At_←{ ⍝ Partial @ - left and right operands must be functions, and the right arg a vector
    data ← ⍵
    (mask/data) ← ⍺⍺ (mask←⍵⍵ ⍵)/⍵
    data
}

In [84]:
{5+⍵}_At_{0=2|⍵} ⍳12

┌→──────────────────────────┐
│5 1 7 3 9 5 11 7 13 9 15 11│
└~──────────────────────────┘

We used leading and trailing underscores when naming our operator to indicate visually that it's a dyad, as suggested by the unofficial [style guide](https://abrudz.github.io/style/#cc).

## Higher rank: choose and reach


The real `@` has many more tricks up its sleeve, of course. It can also be applied to arrays of any rank, not just vectors. 

If we look at [Dyalog's specification of `@`](https://help.dyalog.com/latest/#Language/Primitive%20Operators/At.htm) we have

```apl
R←{X}(f@g)Y
```
    
If `g` is a simple vector, it chooses _major cells_ in Y. If g is nested, it specifies indices for _choose_ or _reach_ indexing. What does that mean, then? One way to think of it is that the g vector behaves just as if it had been inserted between square brackets.

The first case is straight-forward: the major cells of an array is basically those given by the first axis of the shape. In the case of a 2-D matrix, its rows:

In [85]:
-@0 2 ⊢ 3 3⍴1 2 3 4 5 6 7 8 9 ⍝ Negate rows 0 and 2: major cells

┌→───────┐
↓¯1 ¯2 ¯3│
│ 4 5 6│
│¯7 ¯8 ¯9│
└~───────┘

If we want to access individual elements we can use _choose indexing_ -- which is a nested vector of indices:

In [86]:
-@(0 0)(2 2) ⊢ 3 3⍴1 2 3 4 5 6 7 8 9 ⍝ Negate corners of main diagonal by choose indexing

┌→──────┐
↓¯1 2 3│
│ 4 5 6│
│ 7 8 ¯9│
└~──────┘

We can also access elements that are deeply nested using _reach indexing_ which we met briefly in the [indexing](./indexing.ipynb) section:

In [18]:
⊢G←2 3⍴('Adam' 1)('Bob' 2)('Carl' 3)('Danni' 4)('Eve' 5)('Frank' 6)
'***' ¯999@((0 0)0)((1 2)1) ⊢ G

┌─────────┬───────┬─────────┐
│┌────┬─┐ │┌───┬─┐│┌────┬─┐ │
││Adam│1│ ││Bob│2│││Carl│3│ │
│└────┴─┘ │└───┴─┘│└────┴─┘ │
├─────────┼───────┼─────────┤
│┌─────┬─┐│┌───┬─┐│┌─────┬─┐│
││Danni│4│││Eve│5│││Frank│6││
│└─────┴─┘│└───┴─┘│└─────┴─┘│
└─────────┴───────┴─────────┘

┌─────────┬───────┬────────────┐
│┌───┬─┐ │┌───┬─┐│┌────┬─┐ │
││***│1│ ││Bob│2│││Carl│3│ │
│└───┴─┘ │└───┴─┘│└────┴─┘ │
├─────────┼───────┼────────────┤
│┌─────┬─┐│┌───┬─┐│┌─────┬────┐│
││Danni│4│││Eve│5│││Frank│¯999││
│└─────┴─┘│└───┴─┘│└─────┴────┘│
└─────────┴───────┴────────────┘

## Examples

Here's a random collection of handy `@` expressions. Many more on [APLCart](https://aplcart.info/?q=%40#), too.

**Replace dashes with spaces**

In [6]:
' '@(=∘'-')⊢'Hello-World-One-Two'

Hello World One Two

**Pad an array with zeros**

In [5]:
{⍵@(1+⍳⍴⍵)⊢0⍴⍨2+⍴⍵} 2 2⍴1 1 1 1 

0 0 0 0
0 1 1 0
0 1 1 0
0 0 0 0

**Array search and replace**

In [4]:
]DISPLAY array ← ?10 3⍴10
]DISPLAY 1 2 3 4 {⍺⍺(⍵⍵⌷⍨∘⊂⍳)@(∊∘⍺⍺)⍵} 0 10 100 1000 ⊢ array 

0 5 5
8 1 5
1 6 3
5 9 2
5 6 9
0 7 5
3 0 2
4 7 2
9 1 0
0 7 2

0 5 5
 8 0 5
 0 6 100
 5 9 10
 5 6 9
 0 7 5
 100 0 10
1000 7 10
 9 0 0
 0 7 10

**Boolean alternating selection**

Here's a creative use of `@` that [Adám Brudzewsky](https://aplwiki.com/wiki/Ad%C3%A1m_Brudzewsky) posted on [APL Orchard](https://chat.stackexchange.com/transcript/message/57882002#57882002).

Given two arrays A, B and a boolean filter C, select items from A where C is false and from B where C is true:

In [14]:
A←1 2 3 4
B←11 22 33 44
C←0 1 0 1
(C/⍥,B)@{C}A

1 22 3 44

If that looks puzzling (we haven't met the `⍥` operator yet!), for vectors, it's just

In [15]:
(C/B)@{C}A

1 22 3 44

All that says is: replace the true spots (as defined by C) in A with the true spots (as defined by C) from B, which we could also do as an assignable indexing expression if we don't mind mutating A:

In [16]:
(C/A)←C/B

In [17]:
A

1 22 3 44